In [67]:
import pandas as pd
import requests as r
import json

In [68]:
#setting konfigurasi
with open("../env.json", "r", encoding="utf-8") as file:
    config = json.load(file)  # Membaca dan parsing JSON
varId = 1

## Fungsi Utama

In [69]:
#Get Daftar Data Dinamis BPS Bali
def getVarByDomain(domain='5100'):
    global config
    var_pages=[]
    var = r.get(f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/key/{config['token']}/')
    pages=var.json()['data'][0]['pages']
    current_page = 1
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/page/{current_page}/key/{config['token']}/'
        var_page=r.get(url)
        var_pages.append(pd.json_normalize(var_page.json()['data'][1]))
        current_page=current_page+1
    var_pages_df = pd.concat(var_pages)
    return var_pages_df

In [70]:
#Get Daftar Periode
def getPriodeByVar(domain='5100',varId=varId):
    global config
    current_page = 1
    url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
    dt_json = r.get(url)
    pages = dt_json.json()['data'][0]['pages']
    dt_all = []
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
        dt = pd.json_normalize(dt_json.json()['data'][1])
        dt_all.append(dt)
        current_page = current_page+1
    dt_all = pd.concat(dt_all)
    dt_all['varId']=varId
    return dt_all

In [71]:
def crossJoin(left,right):
    if left.empty:
        left = right
    else:
        left = pd.merge(left,right,how='cross')
    return left
def getDataByVarId(varId=varId,domain='5100'):
    global config
    #Banyaknya Wisatawan Mancanegara Bulanan ke Bali Menurut Pintu Masuk
    dt_th = getPriodeByVar(domain=domain,varId=varId)
    dt_all = []
    for i in dt_th.index:
        th_id = dt_th.iloc[i]['th_id']
        url = f'https://webapi.bps.go.id/v1/api/list/model/data/domain/{domain}/var/{varId}/key/{config['token']}/th/{th_id}/'
        dt_json = r.get(url)
        attr = pd.DataFrame()
        if 'vervar' in dt_json.json().keys():
            vervar=pd.json_normalize(dt_json.json()['vervar'])
            vervar=vervar.rename(columns={'val': 'verval_val','label': 'vervar_label'})
            attr = crossJoin(attr,vervar)
        if 'var' in dt_json.json().keys():
            var=pd.json_normalize(dt_json.json()['var'])
            var=var.rename(columns={'val': 'var_val','label': 'var_label'})
            var=var[['var_val']]
            attr = crossJoin(attr,var)
        if 'turvar' in dt_json.json().keys():
            turvar=pd.json_normalize(dt_json.json()['turvar'])
            turvar=turvar.rename(columns={'val': 'turvar_val','label': 'turvar_label'})
            attr = crossJoin(attr,turvar)
        if 'tahun' in dt_json.json().keys():
            tahun=pd.json_normalize(dt_json.json()['tahun'])
            tahun=tahun.rename(columns={'val': 'tahun_val','label': 'tahun_label'})
            attr = crossJoin(attr,tahun)
        if 'turtahun' in dt_json.json().keys():
            turtahun=pd.json_normalize(dt_json.json()['turtahun'])
            turtahun=turtahun.rename(columns={'val': 'turtahun_val','label': 'turtahun_label'})
            attr = crossJoin(attr,turtahun)
        val_columns = [col for col in attr.columns if col.endswith('val')]
        attr['id']=attr[val_columns].astype('str').agg(''.join, axis=1)
        dt_content=pd.json_normalize(dt_json.json()['datacontent']).transpose().reset_index().rename(columns={'index':'content_val',0:'content_label'})
        attr = pd.merge(attr,dt_content,left_on='id',right_on='content_val',how='left')
        attr = attr[attr.content_label.notna()]
        dt_all.append(attr)
    dt_all=pd.concat(dt_all)
    return dt_all

In [90]:
def clean_tag_html(data):
    data.loc[:, "vervar_label"] = data["vervar_label"].str.replace(r"<.*?>", "", regex=True)
    return data
    

In [72]:
daftar_data=getVarByDomain(domain='5100')

### (1) Inflasi Tahunan Kota Denpasar, Singaraja, dan Nasional

In [82]:
data1=getDataByVarId(domain='5100',varId=varId)
data1.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Bahan Makanan,47,0,Tidak ada,119,2019,0,Tahun,14701190,14701190,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",47,0,Tidak ada,119,2019,0,Tahun,24701190,24701190,3.82
2,3,Perumahan,47,0,Tidak ada,119,2019,0,Tahun,34701190,34701190,0.60
3,4,Sandang,47,0,Tidak ada,119,2019,0,Tahun,44701190,44701190,4.02
4,5,Kesehatan,47,0,Tidak ada,119,2019,0,Tahun,54701190,54701190,5.67


In [83]:
#proses cleaning
data1=data1[['verval_val','vervar_label','tahun_label','content_label']]
#proses rename colomn
data1_tb=data1.rename(columns={'vervar_label' : 'wilayah_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data1_tb=data1_tb.pivot_table(index=['verval_val','wilayah_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data1_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data1_tb.columns.values]
data1_tb=data1_tb.reset_index()
data1_tb.drop(columns=['verval_val'])
data1_tb.head()

,verval_val,wilayah_inflasi,inflasi_2014,inflasi_2015,inflasi_2016,inflasi_2017,inflasi_2018,inflasi_2019
0,1,Bahan Makanan,8.09,4.74,9.98,-3.39,4.33,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",15.00,4.22,4.08,3.15,0.77,3.82
2,3,Perumahan,9.97,3.33,1.83,8.44,-3.07,0.60
3,4,Sandang,6.25,7.73,8.47,6.53,4.42,4.02
4,5,Kesehatan,1.45,2.61,4.65,5.02,2.67,5.67


In [75]:
# Export to Excel
data1_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (47) Inflasi Tahunan Kota Singaraja Menurut 7 Kelompok Pengeluaran

In [84]:
varId = 47
data47=getDataByVarId(domain='5100',varId=varId)
data47.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Bahan Makanan,47,0,Tidak ada,119,2019,0,Tahun,14701190,14701190,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",47,0,Tidak ada,119,2019,0,Tahun,24701190,24701190,3.82
2,3,Perumahan,47,0,Tidak ada,119,2019,0,Tahun,34701190,34701190,0.60
3,4,Sandang,47,0,Tidak ada,119,2019,0,Tahun,44701190,44701190,4.02
4,5,Kesehatan,47,0,Tidak ada,119,2019,0,Tahun,54701190,54701190,5.67


In [85]:
#proses cleaning
data47=data47[['verval_val','vervar_label','tahun_label','content_label']]
data47 = clean_tag_html(data47)
#proses rename colomn
data47_tb=data47.rename(columns={'vervar_label' : 'kelompok_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data47_tb=data47_tb.pivot_table(index=['verval_val','kelompok_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data47_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data47_tb.columns.values]
data47_tb=data47_tb.reset_index()
data47_tb.drop(columns=['verval_val'])
data47_tb.head()

,verval_val,kelompok_inflasi,inflasi_2014,inflasi_2015,inflasi_2016,inflasi_2017,inflasi_2018,inflasi_2019
0,1,Bahan Makanan,8.09,4.74,9.98,-3.39,4.33,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",15.00,4.22,4.08,3.15,0.77,3.82
2,3,Perumahan,9.97,3.33,1.83,8.44,-3.07,0.60
3,4,Sandang,6.25,7.73,8.47,6.53,4.42,4.02
4,5,Kesehatan,1.45,2.61,4.65,5.02,2.67,5.67


In [86]:
# Export to Excel
data47_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (65) Inflasi Tahunan Kota Denpasar Menurut 7 Kelompok Pengeluaran

In [91]:
varId = 65
data65=getDataByVarId(domain='5100',varId=varId)
data65.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Bahan Makanan,65,0,Tidak ada,119,2019,0,Tahun,16501190,16501190,0.76
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",65,0,Tidak ada,119,2019,0,Tahun,26501190,26501190,5.98
2,3,Perumahan,65,0,Tidak ada,119,2019,0,Tahun,36501190,36501190,1.34
3,4,Sandang,65,0,Tidak ada,119,2019,0,Tahun,46501190,46501190,2.22
4,5,Kesehatan,65,0,Tidak ada,119,2019,0,Tahun,56501190,56501190,3.16


In [ ]:
#proses cleaning
data65=data65[['verval_val','vervar_label','tahun_label','content_label']]
data65 = clean_tag_html(data65)
#proses rename colomn
data65_tb=data65.rename(columns={'vervar_label' : 'kelompok_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data65_tb=data65_tb.pivot_table(index=['verval_val','kelompok_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data65_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data65_tb.columns.values]
data65_tb=data65_tb.reset_index()
data65_tb.head()

,verval_val,kelompok_inflasi,inflasi_2010,inflasi_2011,inflasi_2012,inflasi_2013,inflasi_2014,inflasi_2015,inflasi_2016,inflasi_2017,inflasi_2018,inflasi_2019
0,1,Bahan Makanan,18.32,3.49,6.06,10.36,11.23,3.31,4.51,1.96,5.86,0.76
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",10.20,5.92,8.67,5.19,6.29,3.55,7.70,6.85,2.39,5.98
2,3,Perumahan,7.31,4.13,3.97,5.64,7.20,5.09,0.39,2.45,1.56,1.34
3,4,Sandang,2.92,6.23,0.92,-1.24,3.60,3.61,5.41,0.32,1.47,2.22
4,5,Kesehatan,1.13,2.41,2.13,1.49,9.99,5.26,3.68,1.48,2.69,3.16


In [93]:
# Export to Excel
data65_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (314) Inflasi Bulanan Kota Denpasar, Singaraja, dan Nasional

In [105]:
varId = 314
data314=getDataByVarId(domain='5100',varId=varId)
data314.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Denpasar,314,0,Tidak ada,123,2023,1,Januari,131401231,131401231,0.62
1,1,Denpasar,314,0,Tidak ada,123,2023,2,Februari,131401232,131401232,0.04
2,1,Denpasar,314,0,Tidak ada,123,2023,3,Maret,131401233,131401233,0.03
3,1,Denpasar,314,0,Tidak ada,123,2023,4,April,131401234,131401234,0.06
4,1,Denpasar,314,0,Tidak ada,123,2023,5,Mei,131401235,131401235,0.34


In [122]:
# proses cleaning
data314 = data314[['verval_val','vervar_label','tahun_label','content_label','turtahun_label']]
data314_tb = data314.rename(columns={
    'vervar_label' : 'kota_inflasi',
    'tahun_label'  : 'tahun',
    'content_label': 'inflasi',
    'turtahun_label':'bulan'
})

data314_tb["periode"] = data314_tb["tahun"].astype(str) + "_" + data314_tb["bulan"].astype(str)
data314_tb = data314_tb.pivot_table(
    index=['verval_val','kota_inflasi'],
    columns=['periode'],
    values=['inflasi']
).reset_index()

data314_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data314_tb.columns.values
]
data314_tb.head()

,verval_val,kota_inflasi,inflasi_2014_Agustus,inflasi_2014_April,inflasi_2014_Desember,inflasi_2014_Februari,inflasi_2014_Januari,inflasi_2014_Juli,inflasi_2014_Juni,inflasi_2014_Maret,...,inflasi_2023_Februari,inflasi_2023_Januari,inflasi_2023_Juli,inflasi_2023_Juni,inflasi_2023_Maret,inflasi_2023_Mei,inflasi_2023_November,inflasi_2023_Oktober,inflasi_2023_September,inflasi_2023_Tahunan
0,1,Denpasar,0.66,0.13,1.99,0.37,1.26,0.49,-0.20,0.32,...,0.04,0.62,0.36,-0.08,0.03,0.34,0.33,0.15,-0.03,2.54
1,2,Singaraja,0.77,0.13,2.80,0.37,0.83,0.39,-0.61,0.17,...,0.29,0.95,0.25,0.22,0.42,0.37,0.87,0.44,-0.05,4.31
2,3,Nasional,0.47,-0.02,2.46,0.26,1.07,0.93,0.43,0.08,...,0.16,0.34,0.21,0.14,0.18,0.09,0.38,0.17,0.19,2.61


In [123]:
data314_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (362) Indeks Harga Konsumen (IHK) Bulanan Kota Denpasar, Singaraja, dan Nasional

In [124]:
varId = 362
data362=getDataByVarId(domain='5100',varId=varId)
data362.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Denpasar,362,0,Tidak ada,123,2023,1,Januari,136201231,136201231,113.87
1,1,Denpasar,362,0,Tidak ada,123,2023,2,Februari,136201232,136201232,113.91
2,1,Denpasar,362,0,Tidak ada,123,2023,3,Maret,136201233,136201233,113.94
3,1,Denpasar,362,0,Tidak ada,123,2023,4,April,136201234,136201234,114.01
4,1,Denpasar,362,0,Tidak ada,123,2023,5,Mei,136201235,136201235,114.40


In [126]:
# proses cleaning
data362 = data362[['verval_val','vervar_label','tahun_label','content_label','turtahun_label']]
data362_tb = data362.rename(columns={
    'vervar_label' : 'kota_inflasi',
    'tahun_label'  : 'tahun',
    'content_label': 'IHK',
    'turtahun_label':'bulan'
})

data362_tb["periode"] = data362_tb["tahun"].astype(str) + "_" + data362_tb["bulan"].astype(str)
data362_tb = data362_tb.pivot_table(
    index=['verval_val','kota_inflasi'],
    columns=['periode'],
    values=['IHK']
).reset_index()

data362_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data362_tb.columns.values
]
data362_tb.head()

,verval_val,kota_inflasi,IHK_2014_Agustus,IHK_2014_April,IHK_2014_Desember,IHK_2014_Februari,IHK_2014_Januari,IHK_2014_Juli,IHK_2014_Juni,IHK_2014_Maret,...,IHK_2023_Desember,IHK_2023_Februari,IHK_2023_Januari,IHK_2023_Juli,IHK_2023_Juni,IHK_2023_Maret,IHK_2023_Mei,IHK_2023_November,IHK_2023_Oktober,IHK_2023_September
0,1,Denpasar,111.42,110.03,116.44,109.54,109.14,110.69,110.15,109.89,...,116.05,113.91,113.87,114.72,114.31,113.94,114.40,115.48,115.10,114.93
1,2,Singaraja,117.70,115.49,125.47,115.10,114.67,116.80,116.35,115.30,...,118.67,115.18,114.85,116.38,116.09,115.66,115.84,118.16,117.14,116.63
2,3,Nasional,113.58,111.35,119.00,111.28,110.99,113.05,112.01,111.37,...,116.56,114.16,113.98,115.24,115.00,114.36,114.84,116.68,115.64,115.44


In [127]:
data362_tb.to_excel(f'output/data_{varId}.xlsx', index=False)